<a href="https://colab.research.google.com/github/okura1406/radnlp-2024/blob/main/radnlp2024_gpt_o1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明
o1モデルを使用してGeminiで比較的良好な結果を出したプロンプトを使用して実行


In [ ]:
from openai import OpenAI
import json
import os
import concurrent.futures
import time

from google.colab import userdata

OPENROUTER_API_KEY=userdata.get('OPEN_ROUTER')

In [ ]:
# エンドポイントを変更
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= OPENROUTER_API_KEY,
)

In [ ]:
# ==============================================
# 1) Google Drive のマウント
# ==============================================
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import pandas as pd
import os

# ==============================================
# 2) CSVファイル (例: label.csv) の読み込み
#    ※パスはご自身の環境に合わせて変更してください
# ==============================================
data_dir = "/content/drive/MyDrive/radnlp_test/test"
valid_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

# valid_df の中身を確認
display(valid_df.head(3))

# ==============================================
# 3) OpenRouter の利用設定
#    (事前にColabの "鍵アイコン" で OPEN_ROUTER を登録しておく)
# ==============================================
!pip install -q openai  # pypiの openai パッケージ (OpenRouterが同一インタフェースを実装)
import json
import time
import requests
import re

from google.colab import userdata
from openai import OpenAI

OPENROUTER_API_KEY = userdata.get("OPEN_ROUTER")

# OpenAIクライアントとしてOpenRouterを使う設定
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

model_name = "openai/o1"

# ==============================================
# 4) レポート本文を取得する関数
# ==============================================
def get_report(report_id):
    """
    report_id (例: ファイル名 "xxx.txt" の 'xxx' 部分) を受け取り
    Google Drive 上から該当txtを読み込んで返す関数
    """
    report_path = os.path.join(data_dir, f"{report_id}.txt")
    with open(report_path, "r", encoding="utf-8") as f:
        report = f.read()
    return report

# ==============================================
# 5) 正規表現で JSON を抽出して Python dict にパースする関数
#    (Gemini時の例と同じ)
# ==============================================
def extract_json(text):
    """
    出力された文章からJSON部分を抜き出し、Python辞書に変換する関数。
    失敗 or 不完全ならデフォルトを返す。
    """
    json_pattern = r'```json\n(.*?)\n```|(\{.*\})'
    matches = re.findall(json_pattern, text, re.DOTALL)

    parsed_json = {}
    for match in matches:
        json_candidate = match[0] if match[0] else match[1]
        try:
            parsed_json = json.loads(json_candidate)
            # JSONパースが成功した時点で抜ける
            break
        except json.JSONDecodeError:
            pass  # 続けて次の候補を試す

    # ※ここで "T"/"N"/"M" が存在しなければデフォルトを補完
    if "T" not in parsed_json:
        parsed_json["T"] = {
            "stage": "T2b",
            "details": "30~50mm程度の腫瘤影"
        }
    if "N" not in parsed_json:
        parsed_json["N"] = {
            "stage": "N0",
            "details": "明らかなリンパ節腫大なし"
        }
    if "M" not in parsed_json:
        parsed_json["M"] = {
            "stage": "M0",
            "details": "遠隔転移を示唆する所見なし"
        }

    return parsed_json


Mounted at /content/drive


,id,t,n,m
0,147290,T0,N0,M0
1,194845,T0,N0,M0
2,221785,T0,N0,M0


In [ ]:

# ==============================================
# 6) 推論実行用のプロンプトを作成
#    (Geminiサンプルの prompt と同等）
# ==============================================
tnm_prompt_header = """「以下の文書は私自身が保有するデータであり、著作権に関する問題はありません。
【instruction】を参考に肺癌について記載された読影レポートをTNM分類で判定し指定の形式にまとめてください。」

【instruction】
{
  "TNM分類_第8版_2017年": {
    "T_原発腫瘍": {
      "TX": "原発腫瘍の存在が判定できない、または喀痰・気管支洗浄液細胞診でのみ陽性で画像・気管支鏡では観察不能",
      "T0": "原発腫瘍を認めない",
      "Tis": "上皮内癌(carcinoma in situ)：肺野型の場合、充実成分径0mmかつ病変全体径≦30mm",
      "T1": {
        "定義": "腫瘍の充実成分径≦30mm、肺または臓側胸膜に覆われ、主気管支への浸潤なし",
        "T1mi": "微小浸潤性腺癌：部分充実型、充実成分径≦5mm、病変全体径≦30mm",
        "T1a": "充実成分径≦10mm、Tis・T1mi以外",
        "T1b": "充実成分径＞10mmかつ≦20mm",
        "T1c": "充実成分径＞20mmかつ≦30mm"
      },
      "T2": {
        "定義": "充実成分径＞30mmかつ≦50mm、または充実成分径≦30mmでも以下の条件を満たす場合:\n1) 主気管支浸潤(気管分岐部は除く)\n2) 臓側胸膜浸潤\n3) 肺門まで連続する無気肺または閉塞性肺炎",
        "T2a": "充実成分径＞30mmかつ≦40mm",
        "T2b": "充実成分径＞40mmかつ≦50mm"
      },
      "T3": {
        "定義": "充実成分径＞50mmかつ≦70mm、または充実成分径≦50mmでも以下のいずれか:\n1) 壁側胸膜、胸壁(上部肺溝腫瘍含む)、横隔神経、心膜への直接浸潤\n2) 同一葉内の不連続な副腫瘍結節"
      },
      "T4": {
        "定義": "充実成分径＞70mm、または以下いずれか:\n1) 横隔膜、縦隔、心臓、大血管、気管、反回神経、食道、椎体、気管分岐部への浸潤\n2) 同側の異なる肺葉内に不連続な副腫瘍結節"
      }
    },
    "N_所属リンパ節": {
      "NX": "所属リンパ節評価不能",
      "N0": "所属リンパ節転移なし",
      "N1": "同側の気管支周囲、肺門、肺内リンパ節転移（原発腫瘍への直接浸潤含む）",
      "N2": "同側縦隔、気管分岐下リンパ節転移",
      "N3": "対側縦隔、対側肺門、前斜角筋・鎖骨上窩リンパ節転移"
    },
    "M_遠隔転移": {
      "M0": "遠隔転移なし",
      "M1": {
        "定義": "遠隔転移あり",
        "M1a": "対側肺内副腫瘍結節、胸膜・心膜結節、悪性胸水・悪性心嚢水（同側・対側問わず）",
        "M1b": "肺以外の一臓器への単発遠隔転移",
        "M1c": "肺以外の一臓器または多臓器への多発遠隔転移"
      }
    }
  },

【T分類判定のステップ】
*腫瘍の有無の確認（最重要）
読影レポートを読み込み、最初に「腫瘍を認めない」、「悪性所見なし」などの明示的な記載の有無を厳格にチェック。
腫瘍を認めない場合 ⇒ T0。
原発腫瘍の存在が判定できない or 細胞診のみ陽性(画像で捉えられない) ⇒ TX。

*大まかなT分類の判定
レポートに “胸壁浸潤” “多発結節” など明らかなキーワードがあれば該当のT(T3/T4)に直行。
なければ 病変の充実成分径、GGO性状、主気管支浸潤の有無などを精査。
胸壁・大血管浸潤など Tを上げる決定的所見(T3/T4 所見)がないか優先チェックし、なければサイズ基準で T1 / T2 を振り分ける。
T1 or T2 と判定した場合、充実成分径の大きさから T1a, T1b, T1c, T2a, T2b に細かく振り分ける。
混同しやすい境界(30mm, 40mm, 50mm) は特に注意。
例: 30mm 付近 ⇒ T1c と T2a の両方を再評価し、他の情報と合わせて最終決定。

*浸潤部位の確認
臓側胸膜 vs 壁側胸膜の記載ゆれにも注意し、壁側胸膜への明確な浸潤があればT3(さらに大きさ次第でT4)とする。
“大血管”“心臓”“椎体”などのキーワード ⇒ T4 を強く疑う。

【Tステージ誤り防止のための重要事項】
1. 早期がん（Tis, T1mi, T0）の過大評価に注意！
充実成分径が非常に小さい(≦5mm)場合は、安易に T1a〜T1c とせず T1mi や Tis の可能性を優先検討。
Tis: 充実成分径0 mm、かつ病変全体30 mm以下。(充実成分径がわずかでも認められる場合はTisの可能性を排除)
T1mi: 部分充実 (mixed GGO) で、solid 部分≦5 mm、全体≦30 mm。(充実成分径が5mmを超えている場合はT1a以降を検討)
画像所見で浸潤が明らかでない場合 ⇒ まず早期がんの可能性を強く意識し、過大評価を避ける。

2. 進行がん（T3, T4）の過小評価に注意！　大径腫瘍・浸潤所見を見落とさない
腫瘍が50mmを超える場合 ⇒ T3 以上を強く検討。
70mm超や胸壁・大血管・心臓などへの浸潤が記載されていればT4の可能性大。
壁側胸膜浸潤、胸壁侵犯、横隔膜・大血管・気管分岐部浸潤などのキーワードを見落とさず、T4を積極的に検討する。

 【判定において特に注意すべき点】
"GGO関連":
純粋GGO(pure GGO)か部分的solidを含むmixed GGOかを明確に抽出し、充実成分の径を厳密に評価。
pure GGO (充実成分0mm)で病変全体30mm以下なら Tis をまず検討。
mixed GGO で 充実成分≦5mm かつ病変全体≦30mm なら T1mi を最優先で検討。

"副腫瘍結節":
同一葉内ならT3、同側の別葉ならT4、対側ならM1a等、位置関係を正しくチェック。
“不連続な結節”か単なる“直接浸潤”かの区別にも留意する。

"主要気管支・縦隔浸潤":
気管分岐部まで及ぶか、胸壁・大血管・横隔膜・心臓などへの直接浸潤があれば、T4を優先的に考慮。

"既知病変と新規原発巣":
レポートで「既知の増悪」「再発疑い」等があれば、T0判定(腫瘍を認めない) の可能性も忘れず検討。


【最終チェック(強制ルール)】
pure GGO が T1c 以上になっていないか 再確認
「腫瘍なし」と明記があれば T0 に矛盾していないか
胸壁・大血管浸潤などのキーワードを再スキャンして T4 を見落としていないか
副腫瘍結節 位置の再チェックで T3/T4/M1a を付け忘れていないか
充実成分径が0mmの純粋GGOがT1a以上になっていないか
充実成分径≦5mmかつ全体径≦30mmのmixed GGOがT1miとして考慮されているか

肺癌のTNM分類について、以下のフォーマットに従って出力してください。
JSON形式で T（原発腫瘍）、N（リンパ節転移）、M（遠隔転移）に関する情報をそれぞれ記載してください。
各値は明確な記載がない場合は最も近いと思われるものとしてください（空欄にならないように各カテゴリは1つの候補に絞ってください）。

{
  "T": {
    "stage": "候補の中から選択",
    "details": "原発腫瘍のサイズや特徴の記載"
  },
  "N": {
    "stage": "候補の中から選択",
    "details": "リンパ節転移の有無や部位の記載"
  },
  "M": {
    "stage": "候補の中から選択",
    "details": "遠隔転移の有無や転移先の記載"
  }
}

【答えの候補】

T（原発腫瘍）:Tis, T1mi, T1a, T1b, T1c, T2a, T2b, T3, T4
N（リンパ節転移）:N0, N1, N2, N3
M（遠隔転移）: M0, M1a, M1b, M1c


【読影レポート】\n
"""


# o1を使用して推論
1分で1リクエスト

In [ ]:

# ==============================================
# 7) 実際にOpenRouterへ問い合わせる関数
# ==============================================
def generate_prediction_with_openrouter(report_text, model=model_name):
    """
    OpenRouter の ChatCompletion API に問い合わせを行い、
    推論結果（JSON含むテキスト）を返す。
    """
    # system, user メッセージを作成
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs the final result in Japanese."
        },
        {
            "role": "user",
            "content": tnm_prompt_header + report_text
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 必要に応じて調整
        max_tokens=30000
    )

    # APIレスポンスから生成結果を取得
    content = response.choices[0].message.content
    return content

# ==============================================
# 8) 推論途中結果をGoogle Driveに保存しながら実行
#    => 中断しても再開できるように改変
# ==============================================
from tqdm import tqdm

# 途中経過CSVファイルのパス(Drive内)
partial_file = "/content/drive/MyDrive/submission_o1　modified_n.csv"

# 既に途中経過ファイルがあれば読み込む
if os.path.exists(partial_file):
    partial_df = pd.read_csv(partial_file)
    # 既に処理済みの id をセット化
    done_ids = set(partial_df["id"].tolist())
    print(f"=== 途中経過ファイルを読み込みました。既に {len(done_ids)} 件処理済みです ===")
else:
    # なければ新規に作る
    partial_df = pd.DataFrame(columns=[
        "id",
        "T_stage", "T_details",
        "N_stage", "N_details",
        "M_stage", "M_details"
    ])
    done_ids = set()
    print("=== 途中経過ファイルは存在しません。新規に作成します ===")

# valid_df を辞書で扱う
rows = valid_df.to_dict(orient="records")

for row in tqdm(rows, desc="Processing rows"):
    report_id = row["id"]

    # すでに処理したレポートならスキップ
    if report_id in done_ids:
        continue

    # レポート取得
    report_text = get_report(report_id)

    # リトライ用
    max_retries = 3
    retries = 0
    generated_text = None

    while retries < max_retries:
        try:
            # OpenRouter推論
            generated_text = generate_prediction_with_openrouter(report_text)
            break
        except (requests.exceptions.RequestException) as e:
            print(f"[Error] report_id={report_id} で失敗: {e}")
            retries += 1
            time.sleep(5)  # 少し待機
            if retries == max_retries:
                print(f"=== 推論リトライ上限に達しました。デフォルト値を格納します。 ===")
                # デフォルト
                default_dict = {
                    "T_stage": "T2b",
                    "T_details": "30~50mm程度の腫瘤影",
                    "N_stage": "N0",
                    "N_details": "明らかなリンパ節腫大なし",
                    "M_stage": "M0",
                    "M_details": "遠隔転移を示唆する所見なし"
                }
                # partial_dfに追記
                new_row = {
                    "id": report_id,
                    **default_dict
                }
                partial_df = pd.concat([partial_df, pd.DataFrame([new_row])], ignore_index=True)
                partial_df.to_csv(partial_file, index=False)
                done_ids.add(report_id)
                # 次のレポートへ
                break

    # 推論が成功した場合のみ
    if generated_text is not None:
        # JSON抽出
        extracted_dict = extract_json(generated_text)

        # partial_dfへ追記
        new_row = {
            "id": report_id,
            "T_stage": extracted_dict["T"]["stage"],
            "T_details": extracted_dict["T"]["details"],
            "N_stage": extracted_dict["N"]["stage"],
            "N_details": extracted_dict["N"]["details"],
            "M_stage": extracted_dict["M"]["stage"],
            "M_details": extracted_dict["M"]["details"]
        }
        partial_df = pd.concat([partial_df, pd.DataFrame([new_row])], ignore_index=True)
        # CSVを都度上書き保存 (再開時に利用できる)
        partial_df.to_csv(partial_file, index=False)
        # 処理済みに追加
        done_ids.add(report_id)

    # API呼び出し間隔（必要に応じて調整）
    time.sleep(60)

print("\n=== 全レポートの処理が完了 or 最終リトライに達しました ===")
print(f"最終的な推論結果は {partial_file} に保存されています。")

# ==============================================
# 9) 任意: partial_df を valid_df と結合して t, n, m カラムへまとめる
# ==============================================
# partial_df には各列:
#   id, T_stage, T_details, N_stage, N_details, M_stage, M_details
# を持っているはず。これをお好みで 1つの DataFrame に統合できます。

# 例: id でマージして、新たな DataFrame (results_df) を作成
results_df = pd.merge(valid_df, partial_df, on="id", how="left")

# t, n, m カラムだけ抜きたい場合は rename してみる
results_df.rename(columns={
    "T_stage": "t",
    "N_stage": "n",
    "M_stage": "m"
}, inplace=True)

# 使いやすいように列順を並べ替え(例)
cols_order = [
    "id", "t", "T_details",
    "n", "N_details",
    "m", "M_details"
]
remaining_cols = [c for c in results_df.columns if c not in cols_order]
results_df = results_df[cols_order + remaining_cols]

display(results_df.head(5))


=== 途中経過ファイルは存在しません。新規に作成します ===


Processing rows: 100%|██████████| 270/270 [6:53:42<00:00, 91.93s/it]


=== 全レポートの処理が完了 or 最終リトライに達しました ===
最終的な推論結果は /content/drive/MyDrive/submission_o1　modified_n.csv に保存されています。


,id,t,t,T_details,n,n,N_details,m,m,M_details
0,147290,T0,T2a,左肺門部に 37mm の腫瘤影を認めるが、胸壁や大血管などへの浸潤所見はなく、主気管支浸潤も...,N0,N0,縦隔リンパ節に有意な腫大や転移を示唆する所見なし,M0,M0,遠隔転移や悪性胸水なし、背部の腫瘤は粉瘤など良性病変を示唆
1,194845,T0,T2b,左肺上葉S1+2に48mmの充実性腫瘤を認め、臓側胸膜への浸潤が示唆されるため(>40mmか...,N0,N0,大動脈弓下に10mmのリンパ節はあるが、病的リンパ節転移を示唆する所見は明確でない,M0,M0,遠隔転移を疑う所見なし
2,221785,T0,T4,右肺尖部の腫瘤が第2・3肋骨および椎体に浸潤し、脊柱管内への進展も疑われるため,N0,N1,右肺門リンパ節が腫大しており転移を疑う所見,M0,M0,遠隔転移を示唆する所見は認めない
3,241752,T0,Tis,右下葉に14×15mmの純粋GGOで充実成分を認めず、病変全体が30mm以内に収まるため,N0,N0,縦隔リンパ節腫大を認めず、リンパ節転移を示唆する所見なし,M0,M0,胸水や腹部臓器への転移性病変を認めず遠隔転移を示唆する所見なし
4,318541,T0,T1mi,左肺下葉S6の病変は全体径12mmで、内部にわずかなsolid成分(5mm以下と推定)を含む...,N0,N0,有意なリンパ節腫大・転移疑い所見なし,M0,M0,遠隔転移を示唆する所見なし


=== マージ後の最終CSVを保存しました: /content/drive/MyDrive/submission_o1_final.csv ===


In [ ]:

# 必要なら最終的なCSVも保存 (別ファイル)
final_output_path = "/content/drive/MyDrive/submission_o1_final.csv"
results_df.to_csv(final_output_path, index=False)
print(f"=== マージ後の最終CSVを保存しました: {final_output_path} ===")

NameError: name 'results_df' is not defined

{"Joint accuracy (fine)":0.7037,<br>"T accuracy (fine)":0.7685, <br>"N accuracy (fine)":0.9306,<br>"M accuracy (fine)":0.9583,<br>"Joint accuracy (coarse)": 	0.7778, <br>"T accuracy (coarse)":0.838, <br>"N accuracy (coarse)":0.9306,<br>"M accuracy (coarse)": 0.9676}